Notebook for Deliverable 2
Getting the data:

Go to https://www.kaggle.com/datasets/rtatman/188-million-us-wildfires?datasetId=2478&sortBy=voteCount
Download dataset by clicking Download in upper-right.
Unzip archive and place in 'data' folder in this repo.

https://www.kaggle.com/code/edhirif/predict-the-causes-of-wildfires-using-python

https://www.kaggle.com/code/captcalculator/can-we-use-ml-to-predict-the-cause-of-a-wildfire

In [37]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#connect to sqlite db
DB_PATH = 'FPA_FOD_20170508.sqlite'
db_connection = sqlite3.connect(DB_PATH)    

In [3]:
df_tables_in_db = pd.read_sql_query("SELECT name FROM sqlite_schema WHERE type='table' ORDER BY name;", db_connection)

In [4]:
# get a list of all the tables in the DB
df_tables_in_db

,name
0,ElementaryGeometries
1,Fires
2,KNN
3,NWCG_UnitIDActive_20170109
4,SpatialIndex
5,geometry_columns
6,geometry_columns_auth
7,geometry_columns_field_infos
8,geometry_columns_statistics
9,geometry_columns_time


In [5]:
# read the Fires table into a dataframe
df_fires = pd.read_sql_query("SELECT * FROM 'Fires'", db_connection)


In [6]:
df_fires.shape

(1880465, 39)

In [7]:
df_fires

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,1880461,300348363,2015CAIRS29019636,NONFED,ST-CACDF,ST/C&L,USCASHU,Shasta-Trinity Unit,CASHU,Shasta-Trinity Unit,...,A,40.481637,-122.389375,13.0,STATE OR PRIVATE,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00P\xb8\x1e\x85\xeb\x9...
1880461,1880462,300348373,2015CAIRS29217935,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,A,37.617619,-120.938570,12.0,MUNICIPAL/LOCAL,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00\x00\x80\xbe\x88\x11...
1880462,1880463,300348375,2015CAIRS28364460,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,A,37.617619,-120.938570,12.0,MUNICIPAL/LOCAL,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00\x00\x80\xbe\x88\x11...
1880463,1880464,300348377,2015CAIRS29218079,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,B,37.672235,-120.898356,12.0,MUNICIPAL/LOCAL,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00x\xba_\xaa~9^\xc0\xb...


In [16]:
df_fires['STAT_CAUSE_DESCR'].unique()

array(['Miscellaneous', 'Lightning', 'Debris Burning', 'Campfire',
       'Equipment Use', 'Arson', 'Children', 'Railroad', 'Smoking',
       'Powerline', 'Structure', 'Fireworks', 'Missing/Undefined'],
      dtype=object)

In [ ]:
./data/

In [ ]:
- FIRE_YEAR = Calendar year in which the fire was discovered or confirmed to exist.
- DISCOVERY_DATE = Date on which the fire was discovered or confirmed to exist.
- DISCOVERY_DOY = Day of year on which the fire was discovered or confirmed to exist.
- DISCOVERY_TIME = Time of day that the fire was discovered or confirmed to exist.
- STATCAUSECODE = Code for the (statistical) cause of the fire.
- STATCAUSEDESCR = Description of the (statistical) cause of the fire.
CONT_DATE = Date on which the fire was declared contained or otherwise controlled (mm/dd/yyyy where mm=month, dd=day, and yyyy=year).
CONT_DOY = Day of year on which the fire was declared contained or otherwise controlled.
CONT_TIME = Time of day that the fire was declared contained or otherwise controlled (hhmm where hh=hour, mm=minutes).
- FIRE_SIZE = Estimate of acres within the final perimeter of the fire.
- FIRESIZECLASS = Code for fire size based on the number of acres within the final fire perimeter expenditures (A=greater than 0 but less than or equal to 0.25 acres, B=0.26-9.9 acres, C=10.0-99.9 acres, D=100-299 acres, E=300 to 999 acres, F=1000 to 4999 acres, and G=5000+ acres).
- LATITUDE = Latitude (NAD83) for point location of the fire (decimal degrees).
- LONGITUDE = Longitude (NAD83) for point location of the fire (decimal degrees).
- STATE = Two-letter alphabetic code for the state in which the fire burned (or originated), based on the nominal designation in the fire report.
COUNTY = County, or equivalent, in which the fire burned (or originated), based on nominal designation in the fire report.

Have wildfires become more or less frequent over time?
What counties are the most and least fire-prone?
Given the size, location and date, can you predict the cause of a fire wildfire?

What level of risk do various geographic areas have for wildfires throughout each year?
Over time, is wildfire frequency and intensity increasing in all areas, or is there variability? Is there a predictable temporal dependence to the increase?
With respect to intensity and frequency, how quickly are different regions able to get fires under control, and how has this changed over time?

fire cause, discovery date and time, containment date and time, fire size (in acres) and class, latitude and longitude, owner/entity responsible for managing land where the fire originated, National Wildfire Coordinating Group unit ID and associated details.


In [8]:
df_fires.columns

Index(['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM',
       'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
       'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME',
       'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY',
       'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE',
       'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE',
       'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY',
       'FIPS_CODE', 'FIPS_NAME', 'Shape'],
      dtype='object')

In [8]:
#Add columns ['DISCOVERY_TIME','CONT_DATE', 'CONT_TIME']
columns_of_interest = ['FIRE_YEAR','STAT_CAUSE_DESCR', 'STAT_CAUSE_CODE','LATITUDE','LONGITUDE','STATE','FIRE_SIZE', 
                       'DISCOVERY_DATE', 'DISCOVERY_TIME','CONT_DATE', 'CONT_TIME']

df_fires[columns_of_interest].head()

,FIRE_YEAR,STAT_CAUSE_DESCR,STAT_CAUSE_CODE,LATITUDE,LONGITUDE,STATE,FIRE_SIZE,DISCOVERY_DATE,DISCOVERY_TIME,CONT_DATE,CONT_TIME
0,2005,Miscellaneous,9.0,40.036944,-121.005833,CA,0.10,2453403.5,1300,2453403.5,1730
1,2004,Lightning,1.0,38.933056,-120.404444,CA,0.25,2453137.5,0845,2453137.5,1530
2,2004,Debris Burning,5.0,38.984167,-120.735556,CA,0.10,2453156.5,1921,2453156.5,2024
3,2004,Lightning,1.0,38.559167,-119.913333,CA,0.10,2453184.5,1600,2453189.5,1400
4,2004,Lightning,1.0,38.559167,-119.933056,CA,0.10,2453184.5,1600,2453189.5,1200


In [10]:
df_fires['DISCOVERY_DATE_Greg'] = pd.to_datetime(df_fires['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
df_fires['CONT_DATE_Greg'] = pd.to_datetime(df_fires['CONT_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
df_fires = df_fires[columns_of_interest+['DISCOVERY_DATE_Greg', 'CONT_DATE_Greg']]
df_fires

,FIRE_YEAR,STAT_CAUSE_DESCR,STAT_CAUSE_CODE,LATITUDE,LONGITUDE,STATE,FIRE_SIZE,DISCOVERY_DATE,DISCOVERY_TIME,CONT_DATE,CONT_TIME,DISCOVERY_DATE_Greg,CONT_DATE_Greg
0,2005,Miscellaneous,9.0,40.036944,-121.005833,CA,0.10,2453403.5,1300,2453403.5,1730,2005-02-02,2005-02-02
1,2004,Lightning,1.0,38.933056,-120.404444,CA,0.25,2453137.5,0845,2453137.5,1530,2004-05-12,2004-05-12
2,2004,Debris Burning,5.0,38.984167,-120.735556,CA,0.10,2453156.5,1921,2453156.5,2024,2004-05-31,2004-05-31
3,2004,Lightning,1.0,38.559167,-119.913333,CA,0.10,2453184.5,1600,2453189.5,1400,2004-06-28,2004-07-03
4,2004,Lightning,1.0,38.559167,-119.933056,CA,0.10,2453184.5,1600,2453189.5,1200,2004-06-28,2004-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,2015,Missing/Undefined,13.0,40.481637,-122.389375,CA,0.01,2457291.5,1726,2457291.5,1843,2015-09-26,2015-09-26
1880461,2015,Miscellaneous,9.0,37.617619,-120.938570,CA,0.20,2457300.5,0126,NaN,None,2015-10-05,NaT
1880462,2015,Missing/Undefined,13.0,37.617619,-120.938570,CA,0.10,2457144.5,2052,NaN,None,2015-05-02,NaT
1880463,2015,Missing/Undefined,13.0,37.672235,-120.898356,CA,2.00,2457309.5,2309,NaN,None,2015-10-14,NaT


In [14]:
# Add month and day of the week
df_fires['DISCOVERY_MONTH'] = pd.DatetimeIndex(df_fires['DISCOVERY_DATE_Greg']).month
df_fires['DISCOVERY_DAY_OF_WEEK'] = df_fires['DISCOVERY_DATE_Greg'].dt.day_name()
df_fires

/var/folders/l4/3ctxtv0s4f10lcsxz1mr2_080000gn/T/ipykernel_4820/697158572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fires['DISCOVERY_MONTH'] = pd.DatetimeIndex(df_fires['DISCOVERY_DATE_Greg']).month
/var/folders/l4/3ctxtv0s4f10lcsxz1mr2_080000gn/T/ipykernel_4820/697158572.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fires['DISCOVERY_DAY_OF_WEEK'] = df_fires['DISCOVERY_DATE_Greg'].dt.day_name()


,FIRE_YEAR,STAT_CAUSE_DESCR,STAT_CAUSE_CODE,LATITUDE,LONGITUDE,STATE,FIRE_SIZE,DISCOVERY_DATE,DISCOVERY_TIME,CONT_DATE,CONT_TIME,DISCOVERY_DATE_Greg,CONT_DATE_Greg,DISCOVERY_MONTH,DISCOVERY_DAY_OF_WEEK
0,2005,Miscellaneous,9.0,40.036944,-121.005833,CA,0.10,2453403.5,1300,2453403.5,1730,2005-02-02,2005-02-02,2,Wednesday
1,2004,Lightning,1.0,38.933056,-120.404444,CA,0.25,2453137.5,0845,2453137.5,1530,2004-05-12,2004-05-12,5,Wednesday
2,2004,Debris Burning,5.0,38.984167,-120.735556,CA,0.10,2453156.5,1921,2453156.5,2024,2004-05-31,2004-05-31,5,Monday
3,2004,Lightning,1.0,38.559167,-119.913333,CA,0.10,2453184.5,1600,2453189.5,1400,2004-06-28,2004-07-03,6,Monday
4,2004,Lightning,1.0,38.559167,-119.933056,CA,0.10,2453184.5,1600,2453189.5,1200,2004-06-28,2004-07-03,6,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,2015,Missing/Undefined,13.0,40.481637,-122.389375,CA,0.01,2457291.5,1726,2457291.5,1843,2015-09-26,2015-09-26,9,Saturday
1880461,2015,Miscellaneous,9.0,37.617619,-120.938570,CA,0.20,2457300.5,0126,NaN,None,2015-10-05,NaT,10,Monday
1880462,2015,Missing/Undefined,13.0,37.617619,-120.938570,CA,0.10,2457144.5,2052,NaN,None,2015-05-02,NaT,5,Saturday
1880463,2015,Missing/Undefined,13.0,37.672235,-120.898356,CA,2.00,2457309.5,2309,NaN,None,2015-10-14,NaT,10,Wednesday


In [16]:
# Compute the numebr of days since discovery of the fire til it get contained
df_fires['DAYS_SPENT'] = (df_fires['CONT_DATE_Greg']-df_fires['DISCOVERY_DATE_Greg']).dt.days

# Compute the number of hours since discovery of the fire til it get contained
df_fires['DISCOVERY_TIME'] = pd.to_numeric(df_fires['DISCOVERY_TIME']) #convert Time to integer
df_fires['CONT_TIME'] = pd.to_numeric(df_fires['CONT_TIME'])
df_fires['HRS_SPENT'] = df_fires['DAYS_SPENT']*24 + (df_fires['CONT_TIME']//100 + df_fires['CONT_TIME']%100/60) - (df_fires['DISCOVERY_TIME']//100 + df_fires['DISCOVERY_TIME']%100/60)

#df = df_fires[columns_of_interest+['DISCOVERY_DATE_Greg', 'CONT_DATE_Greg','DAYS_SPENT','HRS_SPENT']]

/var/folders/l4/3ctxtv0s4f10lcsxz1mr2_080000gn/T/ipykernel_4820/811740408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fires['DAYS_SPENT'] = (df_fires['CONT_DATE_Greg']-df_fires['DISCOVERY_DATE_Greg']).dt.days
/var/folders/l4/3ctxtv0s4f10lcsxz1mr2_080000gn/T/ipykernel_4820/811740408.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fires['DISCOVERY_TIME'] = pd.to_numeric(df_fires['DISCOVERY_TIME']) #convert Time to integer
/var/folders/l4/3ctxtv0s4f10lcsxz1mr2_080000gn/T/ipykernel_4820/811740

In [17]:
df_fires

,FIRE_YEAR,STAT_CAUSE_DESCR,STAT_CAUSE_CODE,LATITUDE,LONGITUDE,STATE,FIRE_SIZE,DISCOVERY_DATE,DISCOVERY_TIME,CONT_DATE,CONT_TIME,DISCOVERY_DATE_Greg,CONT_DATE_Greg,DISCOVERY_MONTH,DISCOVERY_DAY_OF_WEEK,DAYS_SPENT,HRS_SPENT
0,2005,Miscellaneous,9.0,40.036944,-121.005833,CA,0.10,2453403.5,1300.0,2453403.5,1730.0,2005-02-02,2005-02-02,2,Wednesday,0.0,4.500000
1,2004,Lightning,1.0,38.933056,-120.404444,CA,0.25,2453137.5,845.0,2453137.5,1530.0,2004-05-12,2004-05-12,5,Wednesday,0.0,6.750000
2,2004,Debris Burning,5.0,38.984167,-120.735556,CA,0.10,2453156.5,1921.0,2453156.5,2024.0,2004-05-31,2004-05-31,5,Monday,0.0,1.050000
3,2004,Lightning,1.0,38.559167,-119.913333,CA,0.10,2453184.5,1600.0,2453189.5,1400.0,2004-06-28,2004-07-03,6,Monday,5.0,118.000000
4,2004,Lightning,1.0,38.559167,-119.933056,CA,0.10,2453184.5,1600.0,2453189.5,1200.0,2004-06-28,2004-07-03,6,Monday,5.0,116.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,2015,Missing/Undefined,13.0,40.481637,-122.389375,CA,0.01,2457291.5,1726.0,2457291.5,1843.0,2015-09-26,2015-09-26,9,Saturday,0.0,1.283333
1880461,2015,Miscellaneous,9.0,37.617619,-120.938570,CA,0.20,2457300.5,126.0,NaN,NaN,2015-10-05,NaT,10,Monday,NaN,NaN
1880462,2015,Missing/Undefined,13.0,37.617619,-120.938570,CA,0.10,2457144.5,2052.0,NaN,NaN,2015-05-02,NaT,5,Saturday,NaN,NaN
1880463,2015,Missing/Undefined,13.0,37.672235,-120.898356,CA,2.00,2457309.5,2309.0,NaN,NaN,2015-10-14,NaT,10,Wednesday,NaN,NaN


In [22]:
df.to_csv('file1.csv')

In [18]:
df_fires.to_csv('file2.csv')

In [19]:
df = pd.read_csv("file2.csv")
df

/var/folders/l4/3ctxtv0s4f10lcsxz1mr2_080000gn/T/ipykernel_4820/4279301476.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("file2.csv")


,Unnamed: 0,FIRE_YEAR,STAT_CAUSE_DESCR,STAT_CAUSE_CODE,LATITUDE,LONGITUDE,STATE,FIRE_SIZE,DISCOVERY_DATE,DISCOVERY_TIME,CONT_DATE,CONT_TIME,DISCOVERY_DATE_Greg,CONT_DATE_Greg,DISCOVERY_MONTH,DISCOVERY_DAY_OF_WEEK,DAYS_SPENT,HRS_SPENT
0,0,2005,Miscellaneous,9.0,40.036944,-121.005833,CA,0.10,2453403.5,1300.0,2453403.5,1730.0,2005-02-02,2005-02-02,2,Wednesday,0.0,4.500000
1,1,2004,Lightning,1.0,38.933056,-120.404444,CA,0.25,2453137.5,845.0,2453137.5,1530.0,2004-05-12,2004-05-12,5,Wednesday,0.0,6.750000
2,2,2004,Debris Burning,5.0,38.984167,-120.735556,CA,0.10,2453156.5,1921.0,2453156.5,2024.0,2004-05-31,2004-05-31,5,Monday,0.0,1.050000
3,3,2004,Lightning,1.0,38.559167,-119.913333,CA,0.10,2453184.5,1600.0,2453189.5,1400.0,2004-06-28,2004-07-03,6,Monday,5.0,118.000000
4,4,2004,Lightning,1.0,38.559167,-119.933056,CA,0.10,2453184.5,1600.0,2453189.5,1200.0,2004-06-28,2004-07-03,6,Monday,5.0,116.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,1880460,2015,Missing/Undefined,13.0,40.481637,-122.389375,CA,0.01,2457291.5,1726.0,2457291.5,1843.0,2015-09-26,2015-09-26,9,Saturday,0.0,1.283333
1880461,1880461,2015,Miscellaneous,9.0,37.617619,-120.938570,CA,0.20,2457300.5,126.0,NaN,NaN,2015-10-05,NaN,10,Monday,NaN,NaN
1880462,1880462,2015,Missing/Undefined,13.0,37.617619,-120.938570,CA,0.10,2457144.5,2052.0,NaN,NaN,2015-05-02,NaN,5,Saturday,NaN,NaN
1880463,1880463,2015,Missing/Undefined,13.0,37.672235,-120.898356,CA,2.00,2457309.5,2309.0,NaN,NaN,2015-10-14,NaN,10,Wednesday,NaN,NaN


In [24]:
df = df.drop(columns = ['Unnamed: 0','DISCOVERY_DATE','CONT_DATE'])

In [25]:
df.shape

(1880465, 15)

In [26]:
df

,FIRE_YEAR,STAT_CAUSE_DESCR,STAT_CAUSE_CODE,LATITUDE,LONGITUDE,STATE,FIRE_SIZE,DISCOVERY_TIME,CONT_TIME,DISCOVERY_DATE_Greg,CONT_DATE_Greg,DISCOVERY_MONTH,DISCOVERY_DAY_OF_WEEK,DAYS_SPENT,HRS_SPENT
0,2005,Miscellaneous,9.0,40.036944,-121.005833,CA,0.10,1300.0,1730.0,2005-02-02,2005-02-02,2,Wednesday,0.0,4.500000
1,2004,Lightning,1.0,38.933056,-120.404444,CA,0.25,845.0,1530.0,2004-05-12,2004-05-12,5,Wednesday,0.0,6.750000
2,2004,Debris Burning,5.0,38.984167,-120.735556,CA,0.10,1921.0,2024.0,2004-05-31,2004-05-31,5,Monday,0.0,1.050000
3,2004,Lightning,1.0,38.559167,-119.913333,CA,0.10,1600.0,1400.0,2004-06-28,2004-07-03,6,Monday,5.0,118.000000
4,2004,Lightning,1.0,38.559167,-119.933056,CA,0.10,1600.0,1200.0,2004-06-28,2004-07-03,6,Monday,5.0,116.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,2015,Missing/Undefined,13.0,40.481637,-122.389375,CA,0.01,1726.0,1843.0,2015-09-26,2015-09-26,9,Saturday,0.0,1.283333
1880461,2015,Miscellaneous,9.0,37.617619,-120.938570,CA,0.20,126.0,NaN,2015-10-05,NaN,10,Monday,NaN,NaN
1880462,2015,Missing/Undefined,13.0,37.617619,-120.938570,CA,0.10,2052.0,NaN,2015-05-02,NaN,5,Saturday,NaN,NaN
1880463,2015,Missing/Undefined,13.0,37.672235,-120.898356,CA,2.00,2309.0,NaN,2015-10-14,NaN,10,Wednesday,NaN,NaN


In [48]:
# Add month and day of the week
df['CONTAIN_MONTH'] = pd.DatetimeIndex(df['CONT_DATE_Greg']).month
df['CONTAIN_DAY_OF_WEEK'] = df_fires['CONT_DATE_Greg'].dt.day_name()
df

,FIRE_YEAR,STAT_CAUSE_DESCR,STAT_CAUSE_CODE,LATITUDE,LONGITUDE,STATE,FIRE_SIZE,DISCOVERY_TIME,CONT_TIME,DISCOVERY_DATE_Greg,CONT_DATE_Greg,DISCOVERY_MONTH,DISCOVERY_DAY_OF_WEEK,DAYS_SPENT,HRS_SPENT,DISCOVERY YEAR,CONTAIN_MONTH,CONTAIN_DAY_OF_WEEK
0,2005,Miscellaneous,9.0,40.036944,-121.005833,CA,0.10,1300.0,1730.0,2005-02-02,2005-02-02,2,Wednesday,0.0,4.500000,2005,2.0,Wednesday
1,2004,Lightning,1.0,38.933056,-120.404444,CA,0.25,845.0,1530.0,2004-05-12,2004-05-12,5,Wednesday,0.0,6.750000,2004,5.0,Wednesday
2,2004,Debris Burning,5.0,38.984167,-120.735556,CA,0.10,1921.0,2024.0,2004-05-31,2004-05-31,5,Monday,0.0,1.050000,2004,5.0,Monday
3,2004,Lightning,1.0,38.559167,-119.913333,CA,0.10,1600.0,1400.0,2004-06-28,2004-07-03,6,Monday,5.0,118.000000,2004,7.0,Saturday
4,2004,Lightning,1.0,38.559167,-119.933056,CA,0.10,1600.0,1200.0,2004-06-28,2004-07-03,6,Monday,5.0,116.000000,2004,7.0,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,2015,Missing/Undefined,13.0,40.481637,-122.389375,CA,0.01,1726.0,1843.0,2015-09-26,2015-09-26,9,Saturday,0.0,1.283333,2015,9.0,Saturday
1880461,2015,Miscellaneous,9.0,37.617619,-120.938570,CA,0.20,126.0,NaN,2015-10-05,NaT,10,Monday,NaN,NaN,2015,NaN,NaN
1880462,2015,Missing/Undefined,13.0,37.617619,-120.938570,CA,0.10,2052.0,NaN,2015-05-02,NaT,5,Saturday,NaN,NaN,2015,NaN,NaN
1880463,2015,Missing/Undefined,13.0,37.672235,-120.898356,CA,2.00,2309.0,NaN,2015-10-14,NaT,10,Wednesday,NaN,NaN,2015,NaN,NaN


In [27]:
df['DISCOVERY YEAR'] = pd.DatetimeIndex(df_fires['DISCOVERY_DATE_Greg']).year

In [49]:
df['CONTAIN YEAR'] = pd.DatetimeIndex(df_fires['CONT_DATE_Greg']).year

In [46]:
import datetime as dt
df['DISCOVERY_DATE_Greg']= pd.to_datetime(df['DISCOVERY_DATE_Greg'])
df['CONT_DATE_Greg']= pd.to_datetime(df['CONT_DATE_Greg'])
#df['DISCOVERY DATE'] = pd.to_Datetime(df['DISCOVERY_DATE_Greg'])
#df['DISCOVERY DATE'] = df['DISCOVERY_DATE_Greg'].dt.date

In [51]:
df

,FIRE_YEAR,STAT_CAUSE_DESCR,STAT_CAUSE_CODE,LATITUDE,LONGITUDE,STATE,FIRE_SIZE,DISCOVERY_TIME,CONT_TIME,DISCOVERY_DATE_Greg,CONT_DATE_Greg,DISCOVERY_MONTH,DISCOVERY_DAY_OF_WEEK,DAYS_SPENT,HRS_SPENT,DISCOVERY YEAR,CONTAIN_MONTH,CONTAIN_DAY_OF_WEEK,CONTAIN YEAR
0,2005,Miscellaneous,9.0,40.036944,-121.005833,CA,0.10,1300.0,1730.0,2005-02-02,2005-02-02,2,Wednesday,0.0,4.500000,2005,2.0,Wednesday,2005.0
1,2004,Lightning,1.0,38.933056,-120.404444,CA,0.25,845.0,1530.0,2004-05-12,2004-05-12,5,Wednesday,0.0,6.750000,2004,5.0,Wednesday,2004.0
2,2004,Debris Burning,5.0,38.984167,-120.735556,CA,0.10,1921.0,2024.0,2004-05-31,2004-05-31,5,Monday,0.0,1.050000,2004,5.0,Monday,2004.0
3,2004,Lightning,1.0,38.559167,-119.913333,CA,0.10,1600.0,1400.0,2004-06-28,2004-07-03,6,Monday,5.0,118.000000,2004,7.0,Saturday,2004.0
4,2004,Lightning,1.0,38.559167,-119.933056,CA,0.10,1600.0,1200.0,2004-06-28,2004-07-03,6,Monday,5.0,116.000000,2004,7.0,Saturday,2004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,2015,Missing/Undefined,13.0,40.481637,-122.389375,CA,0.01,1726.0,1843.0,2015-09-26,2015-09-26,9,Saturday,0.0,1.283333,2015,9.0,Saturday,2015.0
1880461,2015,Miscellaneous,9.0,37.617619,-120.938570,CA,0.20,126.0,NaN,2015-10-05,NaT,10,Monday,NaN,NaN,2015,NaN,NaN,NaN
1880462,2015,Missing/Undefined,13.0,37.617619,-120.938570,CA,0.10,2052.0,NaN,2015-05-02,NaT,5,Saturday,NaN,NaN,2015,NaN,NaN,NaN
1880463,2015,Missing/Undefined,13.0,37.672235,-120.898356,CA,2.00,2309.0,NaN,2015-10-14,NaT,10,Wednesday,NaN,NaN,2015,NaN,NaN,NaN


In [52]:
df.to_csv('file3.csv')

In [42]:
df = df.drop(columns=['DISCOVERY DATE'])

In [20]:
print(df.isnull().sum())

percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df 

Unnamed: 0                    0
FIRE_YEAR                     0
STAT_CAUSE_DESCR              0
STAT_CAUSE_CODE               0
LATITUDE                      0
LONGITUDE                     0
STATE                         0
FIRE_SIZE                     0
DISCOVERY_DATE                0
DISCOVERY_TIME           882638
CONT_DATE                891531
CONT_TIME                972553
DISCOVERY_DATE_Greg           0
CONT_DATE_Greg           891531
DISCOVERY_MONTH               0
DISCOVERY_DAY_OF_WEEK         0
DAYS_SPENT               891531
HRS_SPENT                988458
dtype: int64


,column_name,percent_missing
Unnamed: 0,Unnamed: 0,0.000000
FIRE_YEAR,FIRE_YEAR,0.000000
STAT_CAUSE_DESCR,STAT_CAUSE_DESCR,0.000000
STAT_CAUSE_CODE,STAT_CAUSE_CODE,0.000000
LATITUDE,LATITUDE,0.000000
LONGITUDE,LONGITUDE,0.000000
STATE,STATE,0.000000
FIRE_SIZE,FIRE_SIZE,0.000000
DISCOVERY_DATE,DISCOVERY_DATE,0.000000
DISCOVERY_TIME,DISCOVERY_TIME,46.937220
